In [1]:
import os
from dotenv import load_dotenv
from llama_index.core import ChatPromptTemplate, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI

load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

MODEL_NAME = "gpt-4o-mini"

SYSTEM_PROMPT = """
事前知識ではなく、常に提供されたコンテキスト情報を使用して質問に回答してください。
回答内でコンテキストを直接参照しないでください。
「コンテキストに基づいて」や「コンテキスト情報は」、またはそれに類するような記述は避けてください。
"""

QA_PROMPT = """
コンテキスト情報は以下の通りです。
---------------------
{context_str}
---------------------
事前知識ではなくコンテキスト情報を使用して、質問に回答してください。
質問：{query_str}
回答："""

REFINE_PROMPT = """
元の回答を (必要な場合のみ) 以下のコンテキストで改良する機会があります。
-----------
{context_msg}
-----------
新しいコンテキストが与えられた場合、元の回答を改良して、質問 {query_str} に適切に回答します。
コンテキストが役に立たない場合は、元の回答を再度出力します。
元の回答：{existing_answer}"""

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=SYSTEM_PROMPT,
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=QA_PROMPT,
    ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=SYSTEM_PROMPT,
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=REFINE_PROMPT,
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

reader = SimpleDirectoryReader(input_dir='data/text')
documents = reader.load_data()
index = VectorStoreIndex.from_documents(documents)

llm = OpenAI(model=MODEL_NAME)
chat_engine = index.as_chat_engine(
    chat_mode="openai",
    llm=llm,
    similarity_top_k=3,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)

while(True):
    message = input("メッセージを入力:")
    if message.strip() == "":
        break
    display(f"質問:{message}")
    # 質問
    response = await chat_engine.astream_chat(message)
    # 回答を表示
    async for token in response.async_response_gen():
        print(token, end="")

print("\n---ご利用ありがとうございました！---")

メッセージを入力: こんにちは！


'質問:こんにちは！'

こんにちは！今日はどんなことをお手伝いできますか？

メッセージを入力: 退職する際の手続きを教えて


'質問:退職する際の手続きを教えて'

退職する際の手続きは以下の通りです。

1. **退職の申し出と届け出**
   - 自己都合退職を希望する場合は、原則として退職希望日の1ヶ月前までに直属の上司に申し出る必要があります。
   - 退職届（書面）の提出が求められ、提出後は退職日や引き継ぎ内容について上司と人事部と協議します。

2. **退職までの流れ**
   - 退職が決まった後は、以下の流れで準備を進めます。
     1. **引き継ぎ**：担当業務の進捗や内容、必要な手順を文書にまとめ、後任者へ引き継ぎます。
     2. **社内備品の返却**：会社の備品（パソコン、スマートフォン、IDカード、制服など）を退職日までに返却します。
     3. **最終面談**：人事部による退職面談が実施され、退職理由や今後のキャリア計画について意見を伺います。

3. **退職日の決定**
   - 退職日については、業務の都合や引き継ぎ内容を考慮して、上司および人事部と話し合いの上で最終決定します。

退職後には、退職証明書の発行や失業保険の手続き、退職金の支払いなどの手続きも行われます。何か他に知りたいことがあれば教えてください！

メッセージを入力: 



---ご利用ありがとうございました！---
